# Get List of US Tech Stocks

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the Stock Analysis screener URL
url = "https://stockanalysis.com/stocks/sector/technology/"

# Fetch the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Locate the table in the screener
table = soup.find("table")

# Extract headers
headers = [header.text for header in table.find_all("th")]

# Extract rows
rows = []
for row in table.find_all("tr")[1:]:  # Skip header row
    cols = [col.text.strip() for col in row.find_all("td")]
    rows.append(cols)

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)
# drop first column
df.drop(df.columns[0], axis=1, inplace=True)
# rename columns
df.columns = ['Symbol', 'Company', 'Market Cap', '% Change', 'Volume', 'Revenue']
df = df.dropna()
# save to csv
df.to_csv('../data/us_tech_stocks_list.csv', index=False)

# Historical Prices from Yahoo Finance

In [18]:
import yfinance as yf
import pandas as pd

# List of US tech stock tickers
tech_stocks = pd.read_csv('../data/us_tech_stocks_list.csv', na_filter=False)['Symbol'].tolist()

# Define the time horizon
start_date = '1999-01-01'
end_date = '2025-01-01'

## Function

In [19]:
# Function to fetch monthly historical stock prices and compute monthly returns
def fetch_monthly_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    hist_data = stock.history(start=start_date, end=end_date, interval='1mo')  # Fetch monthly data
    hist_data['Ticker'] = ticker
    hist_data['Monthly_Return'] = hist_data['Close'].pct_change() # Compute monthly returns
    return hist_data

## Historical data for US Tech Stocks

In [ ]:
# Collect monthly data for all stocks
all_data = []
for ticker in tech_stocks:
    print(f"Fetching monthly data for {ticker}...")
    stock_data = fetch_monthly_stock_data(ticker, start_date, end_date)
    all_data.append(stock_data)

# Combine all monthly stock data
combined_data = pd.concat(all_data)

print("Monthly stock data collection complete!")

In [21]:
# drop Adj Close column
combined_data.drop(columns=['Adj Close'], inplace=True)
# drop rows with missing values
combined_data.dropna(inplace=True)

In [43]:
# save to csv
combined_data.to_csv('../data/monthly_tech_stocks_data.csv', index=True)

## Historical data for XLK Returns

In [23]:
# Fetch historical data for XLK
xlk_data = fetch_monthly_stock_data('XLK', start_date, end_date)
# drop missing values
xlk_data.dropna(inplace=True)

# Save to CSV
xlk_data.to_csv('../data/xlk_returns.csv', index=True)

print("XLK returns data collection complete!")

XLK returns data collection complete!


# Macro Indicators from FRED

In [1]:
from fredapi import Fred
import os

# Retrieve the FRED API key from the environment variable
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

In [28]:
# Define the macroeconomic indicators with their FRED codes
indicators = {
    "3_Month_Treasury_Bill_Yield": "TB3MS",
    "6_Month_Treasury_Bill_Yield": "TB6MS",
    "1_Year_Treasury_Bill_Yield": "GS1",
    "AAA_Bond_Yield": "AAA",
    "BAA_Bond_Yield": "BAA",
    "Oil_Price_WTI": "DCOILWTICO",
    "Inflation_CPI": "CPIAUCSL",
    "Federal_Funds_Rate": "FEDFUNDS",
    "Unemployment_Rate": "UNRATE"
}

# Fetch data for all indicators
data = {}
for name, code in indicators.items():
    print(f"Fetching {name} ({code})...")
    try:
        # Fetch data from FRED
        series = fred.get_series(code)
        data[name] = series
    except Exception as e:
        print(f"Failed to fetch {name}: {e}")

# Combine all data into a single DataFrame
macro_df = pd.DataFrame(data)

Fetching 3_Month_Treasury_Bill_Yield (TB3MS)...
Fetching 6_Month_Treasury_Bill_Yield (TB6MS)...
Fetching 1_Year_Treasury_Bill_Yield (GS1)...
Fetching AAA_Bond_Yield (AAA)...
Fetching BAA_Bond_Yield (BAA)...
Fetching Oil_Price_WTI (DCOILWTICO)...
Fetching Inflation_CPI (CPIAUCSL)...
Fetching Federal_Funds_Rate (FEDFUNDS)...
Fetching Unemployment_Rate (UNRATE)...


In [ ]:
# Resample data to the first day of each month
macro_df = macro_df.resample('MS').mean()

# Convert relevant columns to decimals (e.g., percentage values)
columns_to_convert = [
    "3_Month_Treasury_Bill_Yield",
    "6_Month_Treasury_Bill_Yield",
    "1_Year_Treasury_Bill_Yield",
    "AAA_Bond_Yield",
    "BAA_Bond_Yield",
    "Federal_Funds_Rate",
    "Unemployment_Rate"
]

# Convert percentage columns to decimal format
macro_df[columns_to_convert] = macro_df[columns_to_convert] / 100

# Calculate Inflation Rate as the monthly percentage change in CPI
macro_df["Inflation_Rate"] = macro_df["Inflation_CPI"].pct_change()  # Get relative change
# Drop the original CPI column if no longer needed
macro_df.drop(columns=["Inflation_CPI"], inplace=True)

# calculate oil price change
macro_df['Oil_Price_WTI_change'] = macro_df['Oil_Price_WTI'].pct_change()
# drop oil price column
macro_df.drop(columns=['Oil_Price_WTI'], inplace=True)

# filter start and end date
macro_df = macro_df.loc[start_date:end_date]

print(macro_df.head())

In [30]:
# parse date to datetime
macro_df.index = pd.to_datetime(macro_df.index)
# Save to CSV
macro_df.to_csv("../data/macro_indicators.csv", index=True, index_label='Date')

# Data Preprocessing 

## Stocks data

### Join Stocks data with Macro data

In [1]:
import pandas as pd
import numpy as np

# Load the stock data
stock_data = pd.read_csv('../data/monthly_tech_stocks_data.csv')
# slice string from date
stock_data['Date'] = stock_data['Date'].str.slice(0,10)
# convert date to datetime
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

stock_data.sort_values(by=['Ticker', 'Date'], inplace=True)

In [2]:
# Load macroeconomic data
macro_data = pd.read_csv("../data/macro_indicators.csv", parse_dates=['Date'])
macro_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         313 non-null    datetime64[ns]
 1   3_Month_Treasury_Bill_Yield  312 non-null    float64       
 2   6_Month_Treasury_Bill_Yield  312 non-null    float64       
 3   1_Year_Treasury_Bill_Yield   312 non-null    float64       
 4   AAA_Bond_Yield               312 non-null    float64       
 5   BAA_Bond_Yield               312 non-null    float64       
 6   Federal_Funds_Rate           312 non-null    float64       
 7   Unemployment_Rate            312 non-null    float64       
 8   Inflation_Rate               313 non-null    float64       
 9   Oil_Price_WTI_change         313 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 24.6 KB


In [3]:
# Merge stock data with macroeconomic data
combined_df = pd.merge(stock_data, macro_data, on='Date', how='left')

# Handle missing values
combined_df.ffill(inplace=True)  # Forward-fill missing macroeconomic data
combined_df.dropna(inplace=True)  # Drop rows with remaining missing values

### Feature Engineering

In [4]:
# Feature Engineering
def add_features(df):
    # Rolling 3-month moving average of returns
    df['Rolling_Return_3M'] = df.groupby('Ticker')['Monthly_Return'].rolling(window=3).mean().reset_index(0, drop=True)
    
    # Rolling volatility (standard deviation) of returns over 3 months
    df['Rolling_Volatility_3M'] = df.groupby('Ticker')['Monthly_Return'].rolling(window=3).std().reset_index(0, drop=True)

    # Compute risk premium
    df['Risk_Premium'] = df['Monthly_Return'] - df['3_Month_Treasury_Bill_Yield']

    # Lagged risk premium (1-month lag)
    df['Lagged_Risk_Premium'] = df.groupby('Ticker')['Risk_Premium'].shift(1)
    
    # Drop rows with NaN values caused by rolling calculations
    df.dropna(subset=['Lagged_Risk_Premium', 'Rolling_Return_3M', 'Rolling_Volatility_3M'], inplace=True)
    return df

final_combined_df = add_features(combined_df)

In [5]:
# save to csv
final_combined_df.to_csv('../data/final_tech_stock_data.csv', index=False)

## XLK data

In [6]:
xlk_data = pd.read_csv('../data/xlk_returns.csv')
# slice string from date
xlk_data['Date'] = xlk_data['Date'].str.slice(0,10)
# convert date to datetime
xlk_data['Date'] = pd.to_datetime(xlk_data['Date'])

# merge with macro data
xlk_data = pd.merge(xlk_data, macro_data, on='Date', how='left')
# compute risk premium
xlk_data['Risk_Premium'] = xlk_data['Monthly_Return'] - xlk_data['3_Month_Treasury_Bill_Yield']
# keep only relevant columns
xlk_data = xlk_data[['Date', '3_Month_Treasury_Bill_Yield', 'Monthly_Return', 'Risk_Premium']]

In [7]:
# save to csv
xlk_data.to_csv('../data/final_xlk_data.csv', index=False)